머신 러닝에서 텍스트 분류 연습을 위해 영화 사이트 IMDB의 리뷰 데이터를 자주 사용합니다. 이 데이터는 리뷰에 대한 텍스트와 해당 리뷰가 긍정인 경우는 1, 부정인 경우는 0으로 표시한 레이블로 구성된 데이터입니다.

스탠포드 대학교에서 2011년에 낸 논문에서 이 데이터를 소개했으며, 당시 이 데이터를 훈련 데이터와 테스트 데이터를 50:50 비율로 분할하여 88.89%의 정확도를 얻었다고 소개하고 있습니다.

논문: http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

파이토치에서는 해당 IMDB 리뷰 데이터를 바로 다운로드 할 수 있도록 지원하고 있습니다. 이제 감성 뷴류를 수행하는 모델을 만들겠습니다.

## **1. 셋팅 하기**

In [17]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets
import random

In [2]:
# 랜덤 시드 고정
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
# 하이퍼 파라미터 정의
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [7]:
# device 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("학습할 기기:", DEVICE)

학습할 기기: cpu


참고로 구글의 Colab에서 한다면 '런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU'를 선택하면 USE_CUDA의 값이 True가 되면서 '다음 기기로 학습합니다: cuda'라는 출력이 나옵니다. 즉, GPU로 연산하겠다는 의미입니다. 반면에 '하드웨어 가속기 > None'을 선택하면 USE_CUDA의 값이 False가 되면서 '다음 기기로 학습합니다: cpu'라는 출력이 나옵니다. 즉, CPU로 연산하겠다는 의미입니다.

## **2. 토치텍스트를 이용한 전처리**

### **1) 데이터 로드하기: torchtext.legacy.data**

torchtext.legacy.data의 Field 클래스를 사용하여 영화 리뷰에 대한 객체 TEXT, 레이블을 위한 객체 LABEL을 생성합니다.

In [13]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

- sequential: 데이터셋이 순차적인 데이터셋인가?
- batch_first: 신경망에 입력되는 텐서의 첫번째 차원값을 batch_size로 사용할 것인가?
- lower: 텍스트 데이터 속 모든 알파벳을 소문자로 할 것인가?

레이블은 단순한 클래스를 나타내는 숫자로 순차적 데이터가 아니기에 False를 줍니다. 또한 Apple과 apple을 같은 단어로 인식해야하므로 lower을 True로 하겠습니다.

### **2) 데이터 로드 및 분할하기 : torchtext.legacy.datasets**

torchtext.legacy.datasets을 통해 IMDB 리뷰 데이터를 다운로드할 수 있습니다. 데이터를 다운 받는 동시에 훈련 데이터와 테스트 데이터를 분할하고, 각각 trainset, testset에 저장합니다.

In [16]:
# 전체 데이터를 훈련 데이터와 테스트 데이터를 8:2로 나누기
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [03:07<00:00, 449kB/s]   


텍스트와 레이블이 제대로 저장되었는지 확인하기 위해서 trainset.fields를 통해 trainset이 포함하는 각 요소를 확인해보겠습니다.

In [18]:
print("trainset의 구성 요소 출력 : ", trainset.fields)

trainset의 구성 요소 출력 :  {'text': <torchtext.legacy.data.field.Field object at 0x000001EF7D40D820>, 'label': <torchtext.legacy.data.field.Field object at 0x000001EF7D40D0A0>}


리뷰 데이터가 저장되어져 있는 text 필드와 레이블이 저장되어져 있는 label 필드가 존재합니다. testset.fields도 출력해보겠습니다.

In [19]:
print("testset의 구성 요소 출력 : ", testset.fields)

testset의 구성 요소 출력 :  {'text': <torchtext.legacy.data.field.Field object at 0x000001EF7D40D820>, 'label': <torchtext.legacy.data.field.Field object at 0x000001EF7D40D0A0>}


첫번째 훈련 샘플과 해당 샘플에 대한 레이블을 출력해보겠습니다.

In [20]:
print(vars(trainset[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

- 'text': []에서 대괄호 안에 위치한 단어들이 첫번째 IMDB 리뷰
- 'label': []에서 대괄호 안의 단어가 첫번째 IMDB 리뷰의 레이블 / 'pos' = 'positive'

### **3) 단어 집합 만들기**

이제 단어 집합을 만들어줍니다. 단어 집합이란 중복을 제거한 총 단어들의 집합을 의미합니다.

In [21]:
TEXT.build_vocab(trainset, min_freq=5) # 단어 집합 생성
LABEL.build_vocab(trainset)

위에서 min_freq는 학습 데이터에서 최소 5번 이상 등장한 단어만을 단어 집합에 추가하겠다는 의미입니다. 이때 학습 데이터에서 5번 미만으로 등장한 단어는 Unknown이라는 의미에서 '< unk >'라는 토큰으로 대체됩니다.

단어 집합의 크기와 클래스의 개수를 변수에 저장하고 출력해봅니다. 단어 집합의 크기란 결국 중복을 제거한 총 단어의 개수입니다. 

In [22]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print("단어 집합의 크기: ", vocab_size)
print("클래스의 개수: ", n_classes)

단어 집합의 크기:  46159
클래스의 개수:  2


stoi로 단어와 각 단어의 정수 인덱스가 저장되어져 있는 딕셔너리 객체에 접근할 수 있습니다.

In [24]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x000001EF7D172370>>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             'a': 3,
             'and': 4,
             'of': 5,
             'to': 6,
             'is': 7,
             'in': 8,
             'i': 9,
             'this': 10,
             'that': 11,
             'it': 12,
             '/><br': 13,
             'was': 14,
             'as': 15,
             'for': 16,
             'with': 17,
             'but': 18,
             'on': 19,
             'movie': 20,
             'his': 21,
             'are': 22,
             'not': 23,
             'film': 24,
             'you': 25,
             'have': 26,
             'he': 27,
             'be': 28,
             'at': 29,
             'one': 30,
             'by': 31,
             'an': 32,
             'they': 33,
             'from': 34,
             'all': 35,
             'who': 36,
       

< unk >가 0번 인덱스로 부여되어있고 그 외에도 수많은 단어들이 고유한 정수 인덱스가 부여된 것을 확인할 수 있습니다.

### **4) 데이터 로더 만들기**

훈련 데이터와 테스트 데이터는 분리하였지만, 검증 데이터도 분리해야 합니다. 훈련 데이터를 다시 8:2로 나누어 검증 데이터를 만들겠습니다. 검증데이터는 valset이란 변수에 저장합니다.

In [25]:
trainset, valset = trainset.split(split_ratio=0.8)

정리하면 훈련 데이터는 trainset, 테스트 데이터는 testset, 검증 데이터는 valset에 저장되어있습니다. 

토치텍스트는 모든 텍스트를 배치 처리하는 것을 지원하고, 단어를 인덱스 번호로 대체하는 Bucketlterator를 제공합니다. Bucketlterator는 batch_size, device, shuffle 등의 인자를 받습니다. BATCH_SIZE는 앞서 64로 설정했었습니다.

In [27]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

이제 train_iter, val_iter, test_iter에는 샘플과 레이블이 64개 단위 묶음으로 저장됩니다. 64개씩 묶었을 때, 총 배치의 개수가 몇 개가 되는지 출력해봅시다. 

In [28]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 313
테스트 데이터의 미니 배치의 개수 : 391
검증 데이터의 미니 배치의 개수 : 79


첫번째 미니 배치의 크기를 확인해보겠습니다.

In [29]:
batch = next(iter(train_iter)) # 첫번째 미니배치
print(batch.text.shape)

torch.Size([64, 457])


첫번째 미니배치의 크기는 64 x 457임을 확인할 수 있습니다. 현재 fix_length를 정해주지 않았으므로 미니 배치 간 샘플들의 길이는 전부 상이합니다. 가령 두 번째 미니 배치의 크기를 확인하면 또 다른 것을 확인할 수 있습니다.

In [30]:
batch = next(iter(train_iter)) # 두번째 미니배치
print(batch.text.shape)

torch.Size([64, 903])


두 개의 미니배치를 꺼내서 크기를 확인했으므로 다시 담기 위해 재로드하겠습니다.

In [31]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

## **3. RNN 모델 구현하기**

In [32]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
    
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 상태를 0 벡터로 초기화
        x, _ = self.gru(x, h_0) # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:, -1, :] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t) # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [34]:
# 모델 설계
model = GRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [35]:
# 모델 훈련 함수
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()
        
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [38]:
# 모델 평가 함수
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [39]:
# 모델 학습
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.69 | val accuracy : 52.76
[Epoch: 2] val loss :  0.69 | val accuracy : 51.56
[Epoch: 3] val loss :  0.69 | val accuracy : 52.22
[Epoch: 4] val loss :  0.70 | val accuracy : 49.84
[Epoch: 5] val loss :  0.62 | val accuracy : 66.70
[Epoch: 6] val loss :  0.41 | val accuracy : 81.76
[Epoch: 7] val loss :  0.35 | val accuracy : 85.54
[Epoch: 8] val loss :  0.34 | val accuracy : 86.10
[Epoch: 9] val loss :  0.38 | val accuracy : 85.98
[Epoch: 10] val loss :  0.40 | val accuracy : 85.88


## **4. 마지막 time step의 hidden state 가져오는 것 이해하기**

위에서 GRU의 마지막 time-step의 hidden state를 가져오는 코드를 보겠습니다.

In [40]:
"""
# GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
x, _ = self.gru(x, h_0)  
# (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
h_t = x[:,-1,:] 
"""

'\n# GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)\nx, _ = self.gru(x, h_0)  \n# (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.\nh_t = x[:,-1,:] \n'

(배치 크기, 시퀀스 길이, 은닉 상태의 크기)라는 3차원 텐서가 x에 저장되고, x는 [:, -1, :] 연산을 통해서 (배치 크기, 은닉 상태의 크기)의 텐서로 변환됩니다. 여기서 어떤 일이 벌어졌는제 임의의 텐서를 사용하여 실습해봅시다.

In [41]:
inputs = torch.rand(3, 4, 5) # 임의의 3차원 텐서 생성

In [42]:
print("텐서의 크기 :", inputs.shape)

텐서의 크기 : torch.Size([3, 4, 5])


현재 임의로 생성한 텐서는 (3, 4, 5)의 크기를 가집니다. 이를 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)로 이해해보겠습니다. 다시 말해 3개의 샘플에 대해서 4개의 시점에 대해서 5차원의 은닉 상태가 존재합니다. 텐서의 값을 출력해보겠습니다.

In [43]:
inputs

tensor([[[0.0201, 0.5706, 0.5732, 0.6949, 0.2825],
         [0.8844, 0.0124, 0.4500, 0.6063, 0.3759],
         [0.6036, 0.5584, 0.6196, 0.7123, 0.9885],
         [0.6619, 0.6609, 0.6474, 0.9190, 0.2404]],

        [[0.0074, 0.4659, 0.2264, 0.4641, 0.8252],
         [0.8962, 0.7964, 0.2932, 0.7037, 0.1189],
         [0.1162, 0.8035, 0.3774, 0.3182, 0.7952],
         [0.3687, 0.0804, 0.2903, 0.1778, 0.4458]],

        [[0.9602, 0.6650, 0.2936, 0.8783, 0.7476],
         [0.6762, 0.1738, 0.1500, 0.6426, 0.1625],
         [0.2424, 0.6627, 0.9657, 0.3344, 0.5854],
         [0.9183, 0.1766, 0.8052, 0.3753, 0.7395]]])

이제 [:, -1, :] 연산을 해봅시다.

In [44]:
inputs[:, -1, :]

tensor([[0.6619, 0.6609, 0.6474, 0.9190, 0.2404],
        [0.3687, 0.0804, 0.2903, 0.1778, 0.4458],
        [0.9183, 0.1766, 0.8052, 0.3753, 0.7395]])

3개의 샘플에서 각 마지막 time-step의 hidden state 값만 가져온 것을 볼 수 있습니다. 

In [45]:
print("텐서의 크기 :", inputs[:, -1, :].shape)

텐서의 크기 : torch.Size([3, 5])


텐서의 크기 또한 (배치 크기, 은닉 상태의 크기)가 됩니다.